In [11]:
import requests
import json
import time
import pandas as pd
import numpy as np
from WebCrawlerLibrary import gen_query_url, parse_response, extract_data_from_api, query_start_date
from datetime import datetime, timedelta

In [12]:
# Design dataframe
API_COLUMNS = ["Title", "Description", "NewsURL", "PublishedTime", "SourceName" , "SourceURL"]

api_content = pd.read_csv("../Data/NewsContent.csv", index_col=[0])
# api_content = pd.read_csv("../Data/NewsContent.csv")

In [13]:
api_content_current_session = pd.DataFrame(columns=API_COLUMNS)

In [14]:
# query_index_date.strftime(datetime_format) != datetime.now().strftime(datetime_format)

In [15]:
# Process query days pipeline
datetime_format = "%Y-%m-%d"
# start_date = datetime(2020,1,29)
start_date = query_start_date(api_content)
# end_date =  datetime(2020,2,1)

query_index_date = start_date 

status_code = 200
counter = 0
max_count = 49
first_time = False


while counter < max_count and status_code == 200 and query_index_date.strftime(datetime_format) != datetime.now().strftime(datetime_format):
    # counter does not surpass 100 queryies per day
    # status_code is still successful
    # index date has yet to surpass today
    
    # Generate URLs
    query_index_date_next = query_index_date + timedelta(days=1)
    url = gen_query_url(query_index_date.strftime(datetime_format), query_index_date_next.strftime(datetime_format) )
    
    # Parse URLs
    response = requests.get(url)
    
    # get Status_code and content
    status_code, output_content = parse_response(response)
    
    # Parse content
    if status_code == 200 and output_content != "no returns":    
        new_api_data = extract_data_from_api(output_content)
    
    if counter == 0 and first_time == True:
        api_content = pd.DataFrame(new_api_data, columns = API_COLUMNS)
#         print(api_content)
    else :
        new_api_df = pd.DataFrame(new_api_data, columns = API_COLUMNS)
#         print(new_api_df)
        api_content = api_content.append(new_api_df, ignore_index = True).drop_duplicates()
        api_content_current_session = api_content_current_session.append(new_api_df, ignore_index = True).drop_duplicates()
    

    # Post Processing conditions
    query_index_date = query_index_date_next
    counter += 1
    if counter == max_count:
        print("Has Reached Max Count of " + str(counter))
    if status_code != 200:
        print("Status Code " + str(status_code))
    time.sleep(5)

https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-08-10&maxdate=2020-08-11&in=title
https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-08-11&maxdate=2020-08-12&in=title


In [16]:
# temp = api_content_current_session.copy()
# temp = temp.drop_duplicates(["Title" , "PublishedTime"])
# api_content = api_content.append(temp, ignore_index = True).drop_duplicates()


In [17]:
api_content.PublishedTime   = pd.to_datetime(api_content.PublishedTime, utc=True).dt.strftime('%Y-%m-%d')
api_content_current_session.PublishedTime   = pd.to_datetime(api_content_current_session.PublishedTime, utc=True).dt.strftime('%Y-%m-%d')

api_content = api_content.drop_duplicates(["Title", "PublishedTime"]).sort_values(["PublishedTime"])
api_content

,Title,Description,NewsURL,PublishedTime,SourceName,SourceURL
0,White House doesn't rule out suspending US-Chi...,The White House is not ruling out suspending f...,https://abcnews.go.com/Politics/white-house-co...,2020-01-29,ABC,https://abcnews.go.com
1,Coronavirus Crisis In China To Have Economic I...,“China controls almost 100% of the generic med...,https://dfw.cbslocal.com/2020/01/29/coronaviru...,2020-01-29,CBS Boston / WBZ,https://dfw.cbslocal.com
2,Coronavirus cases in US should not cause mass ...,A deadly coronavirus that's killed at least 13...,https://www.foxnews.com/health/coronavirus-cas...,2020-01-29,Fox News,https://www.foxnews.com
3,US learned from Ebola but is 'nowhere near as ...,"“The Department of Homeland Security said, ‘Ac...",https://www.usatoday.com/story/news/health/202...,2020-01-29,USA Today,https://www.usatoday.com
4,Coronavirus: Japan and US fly citizens home as...,Japan and US airlift hundreds out of Wuhan as ...,https://www.theguardian.com/world/2020/jan/29/...,2020-01-29,The Guardian,https://www.theguardian.com
...,...,...,...,...,...,...
1651,Health officials are quitting or getting fired...,"Vilified, threatened with violence and in some...",https://nypost.com/2020/08/11/health-officials...,2020-08-11,New York Post,https://nypost.com
1652,Coronavirus: White House weighs plan to keep o...,President Donald Trump has often touted his tr...,https://www.bbc.com/news/world-us-canada-53740840,2020-08-11,BBC,https://www.bbc.com
1653,Coronavirus US: Over 40 members of family infe...,"Polly Williams, of Kermit, West Virginia, says...",https://www.dailymail.co.uk/news/article-86158...,2020-08-11,Daily Mail,https://www.dailymail.co.uk
1654,A Navy spouse and psychotherapist on why the U...,In a country that lacks any disciplined and un...,https://www.businessinsider.com/navy-spouse-on...,2020-08-11,Business Insider,https://www.businessinsider.com


In [18]:
api_content_current_session.to_csv("../Data/NewContentCurrentSession.csv")
api_content.to_csv("../Data/NewsContent.csv")

---

In [19]:

# import GetOldTweets3 as gt

https://ieee-dataport.org/open-access/coronavirus-covid-19-tweets-dataset

In [20]:
# temp1 = temp_cur_session.drop_duplicates(["Title","PublishedTime"])
# api_content_current_session = temp1
# temp_cur_session